## RNN 神經網路

情感分類：
1. 核心情緒: 最直接的感受情緒，如喜怒哀樂等等
2. 抑制情緒: 壓抑對情緒的感受，如焦慮、內疚後悔、慚愧
3. 防禦情緒: 逃避對情緒的感受，如拖延、成癮、疏離


<img src = "/Users/chiachenhsu/Desktop/nlp final project_許家禎/project 2/tri_emotions_pic.jpeg" height="700" width="500">

應用：分析以上三種情緒可以幫助了解個體的情緒變化，透過辨別逃避與壓抑的情感（抑制情緒、防禦情緒）才有辦法掌握掌握自己的真實情緒（核心情緒）。
\
相關應用或許可以包涵分析青少年在網路上的貼文內容，並在他們表現出抑制或防禦情緒時適時給予協助。

語料蒐集：dcard心情版貼文、ptt sad版



文本數量

In [1]:
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def unify_hash_symbols(pos_text):
    return pos_text.replace('＃', '#')

core_emo_path = '/Users/chiachenhsu/Desktop/nlp final project_許家禎/project 2/神經網路語料/核心情緒（喜怒哀樂）.txt'
depress_emo_path = '/Users/chiachenhsu/Desktop/nlp final project_許家禎/project 2/神經網路語料/抑制情緒（慚愧、焦慮、內疚）.txt'
defense_emo_path = '/Users/chiachenhsu/Desktop/nlp final project_許家禎/project 2/神經網路語料/防禦情緒（轉移話題、開玩笑、疏離拖延、上癮.txt'

core_emo_corpus = read_text_file(core_emo_path)
depress_emo_corpus = read_text_file(depress_emo_path)
defense_emo_corpus = read_text_file(defense_emo_path)

core_emo_corpus = unify_hash_symbols(core_emo_corpus)
depress_emo_corpus = unify_hash_symbols(depress_emo_corpus)
defense_emo_corpus = unify_hash_symbols(defense_emo_corpus)

core_texts = [text for text in core_emo_corpus.split('#') if text.strip()]
depress_texts = [text for text in depress_emo_corpus.split('#') if text.strip()]
defense_texts = [text for text in defense_emo_corpus.split('#') if text.strip()]

num_core = len(core_texts)
num_depress = len(depress_texts)
num_defense = len(defense_texts)

print("核心情緒文本數", num_core)
print("抑制情緒文本數", num_depress)
print("防禦情緒文本數", num_defense)


核心情緒文本數 178
抑制情緒文本數 133
防禦情緒文本數 123


## 步驟一：製作斷詞

In [2]:
from utilities import concatFiles, word_segmenter
import pandas as pd 

core_emo, depress_emo, defense_emo = concatFiles('/Users/chiachenhsu/Desktop/nlp final project_許家禎/project 2/神經網路語料')

core_emo_tokens= word_segmenter (core_emo)
depress_emo_tokens= word_segmenter (depress_emo)
defense_emo_tokens= word_segmenter (defense_emo)

core_label= [0.]*len(core_emo_tokens) #核心情緒標0
depress_label= [1.]*len(depress_emo_tokens) #抑制情緒標1
defense_label= [2.]*len(defense_emo_tokens) #防禦情緒標2


all_tokens= core_emo_tokens+ depress_emo_tokens+defense_emo_tokens
all_labels= core_label+depress_label+defense_label

pd.DataFrame({'tokens':all_tokens, 'labels':all_labels}).to_csv('神經網路_斷詞.csv', index=False)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
2024-06-20 13:54:25.468634: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


## 步驟二：製作w2v模型

In [3]:
import os
import pandas as pd
from gensim.models import Word2Vec

dataDF = pd.read_csv('/Users/chiachenhsu/Desktop/nlp final project_許家禎/project 2/神經網路_斷詞.csv')
labels = dataDF['labels'].values

tokensLST = []
for i in range(len(dataDF)):
    tokensLST.append(eval(dataDF.iloc[i]['tokens']))


w2v_model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)
w2v_model.build_vocab(tokensLST)

model_dir = '/Users/chiachenhsu/Desktop/nlp final project/project 2'
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, '神經網路_word2vec.model')
w2v_model.save(model_path)

## 步驟三：讀入斷詞與模型並轉成數值

In [4]:
rnn_dataDF = pd.read_csv("/Users/chiachenhsu/Desktop/nlp final project_許家禎/project 2/神經網路_斷詞.csv")
rnn_labels = rnn_dataDF['labels']

tokenslist = []
for i in range(len(dataDF)):
    tokenslist.append(eval(dataDF.iloc[i]['tokens']))

rnn_model_name = '/Users/chiachenhsu/Desktop/nlp final project/project 2/神經網路_word2vec.model'
rnn_model = Word2Vec.load(rnn_model_name)

vectorized_corpus = []

for tokensTXT in tokenslist:
    temp_vec = []
    for token in tokensTXT:
        if token in rnn_model.wv:
            temp_vec.append(rnn_model.wv[token])
    vectorized_corpus.append(temp_vec)


## 步驟四：RNN訓練

In [5]:
from keras.layers import Dense, Dropout, Flatten, SimpleRNN
from tensorflow.keras.utils import pad_sequences, to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.models import Sequential
import tensorflow as tf 

num_neurons = 30
maxlen = 300
batch_size = 32
embedding_dims = 100
filters = 150 
kernel_size = 3
hidden_dims = 250 
epochs = 5

zero_vec = [1e-9]*embedding_dims
rnn_X = pad_sequences(vectorized_corpus, maxlen=maxlen, dtype='float', padding='post', truncating='post', value=zero_vec)
num_classes= 3
rnn_y = to_categorical(all_labels, num_classes)

X_train, X_test, y_train, y_test = train_test_split(rnn_X, rnn_y, test_size=0.2, random_state=1)

y_train = np.array(y_train)
y_test = np.array(y_test)


rnn_model = Sequential()
rnn_model.add(SimpleRNN(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims))) 
rnn_model.add(Dropout(0.2))
rnn_model.add(Flatten())
rnn_model.add(Dense(3, activation="softmax"))

rnn_model.compile(optimizer="rmsprop", loss='categorical_crossentropy',
              metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),
                         tf.keras.metrics.F1Score(), 'accuracy'])

rnn_model.fit(X_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/5
23/23 [==============================] - 1s 28ms/step - loss: 1.1214 - precision: 0.3425 - recall: 0.0701 - f1_score: 0.3128 - accuracy: 0.3590 - val_loss: 1.1150 - val_precision: 0.2500 - val_recall: 0.0056 - val_f1_score: 0.1967 - val_accuracy: 0.3855
Epoch 2/5
23/23 [==============================] - 0s 21ms/step - loss: 1.0876 - precision: 0.4286 - recall: 0.0252 - f1_score: 0.2925 - accuracy: 0.4053 - val_loss: 1.1165 - val_precision: 0.2500 - val_recall: 0.0391 - val_f1_score: 0.2130 - val_accuracy: 0.4022
Epoch 3/5
23/23 [==============================] - 0s 18ms/step - loss: 1.0783 - precision: 0.4576 - recall: 0.0379 - f1_score: 0.3137 - accuracy: 0.4306 - val_loss: 1.1041 - val_precision: 0.3750 - val_recall: 0.0168 - val_f1_score: 0.2068 - val_accuracy: 0.3855
Epoch 4/5
23/23 [==============================] - 1s 28ms/step - loss: 1.0542 - precision: 0.6667 - recall: 0.0337 - f1_score: 0.3228 - accuracy: 0.4460 - val_loss: 1.4448 - val_precision: 0.3593 - val_recal

訓練集評估指標
|Precision|Recal|F1 score|Accuracy|
|---|---|---|---|
|0.5714|0.1122|0.3770|0.4642|

測試集評估指標
|Precision|Recal|F1 score|Accuracy|
|---|---|---|---|
|0.2143|0.0168|0.3646|0.3799|

評估指標的表現皆不好

## 步驟五：預測新語料
        

In [6]:
from utilities import segmentAndW2V
import torch
device = torch.device('cpu')

#預期結果應為核心情緒，和測試結果不一樣
newEmotion1= ''' 
前幾天心情很不好，於是在凌晨四點到wootalk找人聊天，遇到了一個很好很好的人，他開導了我很多，告訴我一定要活著要請他吃牛排，後來我們加了賴，聊了很久，我還以為找到了知己（我知道自己想太多），不過還是謝謝這個陌生人，如果你有看到，我想說，謝謝你在我最黑暗的時候拉住了我，雖然也許沒有交集了，也不知道為什麼你突然沒找我了，也許是我長得不好看？哈哈哈哈
但我卻有點動心了⋯⋯
相信我可以很快走出來的'''

#預期結果應為抑制情緒，和測試結果一樣
newEmotion2 = '''
最近發現自己常常覺得心情低落、心裡莫名煩躁 然後什麼事都不想做
要脫離這種感覺只有看綜藝或追劇煩躁感才比較容易消失
如果說要出去外面逛逛也不好說
最近因為被騙錢 所以想報復性花錢也只能想想（就算沒被騙也不太可能）
台北的物價大家也知道 就算什麼都不買只吃東西也不便宜
我是屬於不好的情緒不太會表現出來 都自己默默消化的那種 所以外人眼裡我幾乎都是很正向樂觀
想問如果有遇到這種心情的都是怎麼調適
請留言的各位手下留情
太多一言不合就吵的有點恐怖'''

#預期結果應為防禦情緒，和測試結果一樣
newEmotion3 = '''心情好糟
發瘋
找不到適合求助
每次總是吃虧
越來越恨這世界
沒人在乎..'''


newEmotions = [newEmotion1, newEmotion2, newEmotion3]
newEmotionsMAT = segmentAndW2V(newEmotions)


preds = rnn_model.predict(newEmotionsMAT)
print(preds)
print()

categories = {0: '核心情緒', 1: '抑制情緒', 2: '防禦情緒'}

for i, txt in enumerate(newEmotions):
    print(f"要預測的貼文為：\n{txt}\n")
    
    pred_label = preds[i].argmax()
    print(f"這則貼文表現的情緒為：{categories[pred_label]}")
    print()



Inference: 100%|██████████| 1/1 [00:00<00:00,  1.86it/s]

1/1 [==============================] - 0s 72ms/step
[[0.28247163 0.3778495  0.33967882]
 [0.33318555 0.38945654 0.2773579 ]
 [0.3980269  0.25898156 0.34299153]]

要預測的貼文為：
 
前幾天心情很不好，於是在凌晨四點到wootalk找人聊天，遇到了一個很好很好的人，他開導了我很多，告訴我一定要活著要請他吃牛排，後來我們加了賴，聊了很久，我還以為找到了知己（我知道自己想太多），不過還是謝謝這個陌生人，如果你有看到，我想說，謝謝你在我最黑暗的時候拉住了我，雖然也許沒有交集了，也不知道為什麼你突然沒找我了，也許是我長得不好看？哈哈哈哈
但我卻有點動心了⋯⋯
相信我可以很快走出來的

這則貼文表現的情緒為：抑制情緒

要預測的貼文為：

最近發現自己常常覺得心情低落、心裡莫名煩躁 然後什麼事都不想做
要脫離這種感覺只有看綜藝或追劇煩躁感才比較容易消失
如果說要出去外面逛逛也不好說
最近因為被騙錢 所以想報復性花錢也只能想想（就算沒被騙也不太可能）
台北的物價大家也知道 就算什麼都不買只吃東西也不便宜
我是屬於不好的情緒不太會表現出來 都自己默默消化的那種 所以外人眼裡我幾乎都是很正向樂觀
想問如果有遇到這種心情的都是怎麼調適
請留言的各位手下留情
太多一言不合就吵的有點恐怖

這則貼文表現的情緒為：抑制情緒

要預測的貼文為：
心情好糟
發瘋
找不到適合求助
每次總是吃虧
越來越恨這世界
沒人在乎..

這則貼文表現的情緒為：核心情緒



## 綜合討論

第一則貼文：分類錯誤。作者提到相信自己很快釋懷，是核心情緒的定義，但是卻誤判成抑制情緒。
\
\
第二則貼文：分類正確。文本提及低落與煩躁，和抑制情緒中焦慮的特徵類似。
\
\
第三則貼文：分類錯誤。原文提到提到「找不到求助」、「沒人在乎」，為防禦情緒的疏離態度展現，但是卻誤判成核心情緒。


## 後續修改
由於評估指標數值欠佳，將進行以下修改看能否提升分類結果

1. 修改一：自製的w2v模型因語料太少，表現不好，所以改用課堂使用的ccu_w2v_model
2. 修改二：修改超參數與調整dropout層，提升評估指標表現

修改一：斷詞、導入ccu_w2v_model模型、將斷詞轉成數值

In [7]:
rnn_dataDF = pd.read_csv("/Users/chiachenhsu/Desktop/nlp final project_許家禎/project 2/神經網路_斷詞.csv")
rnn_labels = rnn_dataDF['labels']

tokenslist = []
for i in range(len(dataDF)):
    tokenslist.append(eval(dataDF.iloc[i]['tokens']))

ccu_model_name= '/Users/chiachenhsu/Desktop/nlp final project_許家禎/project 2/ccu_w2v_model04'
ccu_model = Word2Vec.load(ccu_model_name)

vectorized_corpus = []

for tokensTXT in tokenslist:
    temp_vec = []
    for token in tokensTXT:
        if token in ccu_model.wv:
            temp_vec.append(ccu_model.wv[token])
    vectorized_corpus.append(temp_vec)

修改二：修改超參數與調整dropout層

In [9]:
from keras.layers import Dense, Dropout, Flatten, SimpleRNN
from tensorflow.keras.utils import pad_sequences, to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.models import Sequential
import tensorflow as tf 

num_neurons = 50 #增加神經元數
maxlen = 300
batch_size = 32
embedding_dims = 300
filters = 200 
kernel_size = 3
hidden_dims = 250 
epochs = 20 #提高迭代次數

zero_vec = [1e-9]*embedding_dims
rnn_X = pad_sequences(vectorized_corpus, maxlen=maxlen, dtype='float', padding='post', truncating='post', value=zero_vec)
num_classes= 3
rnn_y = to_categorical(all_labels, num_classes)

X_train, X_test, y_train, y_test = train_test_split(rnn_X, rnn_y, test_size=0.2, random_state=1)

y_train = np.array(y_train)
y_test = np.array(y_test)


rnn_model = Sequential()
rnn_model.add(SimpleRNN(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims))) 
rnn_model.add(SimpleRNN(num_neurons, return_sequences=True))  
rnn_model.add(SimpleRNN(num_neurons, return_sequences=True))  #增加層數
rnn_model.add(Dropout(0.35)) #提高dropout
rnn_model.add(Flatten())
rnn_model.add(Dense(3, activation="softmax"))

rnn_model.compile(optimizer="adam", loss='categorical_crossentropy', #把優化器從rmsprop改成adam
              metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),
                         tf.keras.metrics.F1Score(), 'accuracy'])

rnn_model.fit(X_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/20
23/23 [==============================] - 3s 87ms/step - loss: 1.3500 - precision_2: 0.3849 - recall_2: 0.2791 - f1_score: 0.3744 - accuracy: 0.3913 - val_loss: 1.7713 - val_precision_2: 0.3860 - val_recall_2: 0.3687 - val_f1_score: 0.3017 - val_accuracy: 0.3799
Epoch 2/20
23/23 [==============================] - 2s 74ms/step - loss: 1.1249 - precision_2: 0.5529 - recall_2: 0.4544 - f1_score: 0.4921 - accuracy: 0.5077 - val_loss: 1.4377 - val_precision_2: 0.2909 - val_recall_2: 0.2682 - val_f1_score: 0.2848 - val_accuracy: 0.2849
Epoch 3/20
23/23 [==============================] - 2s 74ms/step - loss: 0.8787 - precision_2: 0.6470 - recall_2: 0.5063 - f1_score: 0.5848 - accuracy: 0.5975 - val_loss: 1.4639 - val_precision_2: 0.3312 - val_recall_2: 0.2849 - val_f1_score: 0.3289 - val_accuracy: 0.3408
Epoch 4/20
23/23 [==============================] - 2s 81ms/step - loss: 0.8393 - precision_2: 0.6490 - recall_2: 0.5498 - f1_score: 0.6006 - accuracy: 0.6045 - val_loss: 1.2909 - v

## 修改前後比較

訓練集
|評估指標|Precision|Recal|F1 score|Accuracy|
|---|---|---|---|---|
|修改前|0.5714|0.1122|0.3770|0.4642|
|修改後|0.7719|0.6171|0.6704|0.6802|

測試集
|評估指標|Precision|Recal|F1 score|Accuracy|
|---|---|---|---|---|
|修改前|0.2143|0.0168|0.3646|0.3799|
|修改後|0.3622|0.2570|0.3198|0.3296|

## 結語

評估指標表現整體而言仍偏低，但透過以下修改仍可以微幅提升分類的表現。包含：
1. 改用語料數較龐大的w2v模型
2. 增加神經元數: 原30增至50
3. 增加迭代次數: 原5次增至20次
4. 提高dropout:原0.2提高到0.35
5. 增加神經網路層數: 原1層增至3層
6. 不同優化器：原rmsprop改成adam